In [127]:
import json
import os

import shutil
import base64
import requests
import pandas as pd 

from dotenv import load_dotenv
import google.generativeai as genai

In [128]:
# fdata = []
# data = []
# with open("dataset/chitron-captions-fresh.json", "r+") as f:
#     data = json.load(f)

In [308]:
# df = pd.json_normalize(data)
# df.loc[:,'Generated'] = None

df = pd.read_csv("dataset/generated_chitron.csv")

In [309]:
# df.to_csv("dummy.csv", index=False)
df.head()

,filename,caption,Generated,longest_caption
0,1.png,['তিন জন মেয়ে মানুষ আছে। এক জন দাড়িয়ে আছে আর দ...,"Q# ""ছবিতে মহিলাটির হাতে কি আছে?"", A# ""বেত""",'একটি হলুদ জামা পায়জামা পরা মহিলা দাড়িয়ে হাতে...
1,2.png,['জলাশয় এ একজন ছেলে মানুষ ও একটি বাচ্চা মানুষ ...,"Q# ""নৌকায় দাঁড়ানো পুরুষটি কি পরে আছেন?"", A# ""স...",'২ জন মানুষ যার একজন পুরুষ একটি নৌকায় দাড়িয়ে ...
2,3.png,"['অনেক মেয়ে মানুষ বসে আছে।', 'একটি নীল জামা পর...","Q# ""ল্যাপটপ ধারণকারী মহিলার জামার রঙ কি?"", A# ...",'একটি নীল জামা পরা মহিলা একটি নীল ল্যাপটপ এর ...
3,4.png,"['অনেক মানুষ একসাথে বসে কাজ করছে।', '২ টি ছোট...","Q# ""ছবিতে বসে থাকা ছোট ছেলেটি কী ধরনের পোশাক প...",'২ টি ছোট ছেলে একজন শার্ট প্যান্ট দাড়িয়ে চে...
4,5.png,"['ছয় জন মানুষ দাড়িয়ে আছে।', '৬ জন মানুষ এলোমেল...","Q# ""এই ছবিতে কতজন পুরুষ লুঙ্গী পরে আছেন?""\nA# ...","'৬ জন মানুষ এলোমেলো দাড়িয়ে আছে, তাদের মাঝে ২ ..."


In [310]:
# validation = df[mask]
df.isna().sum()

filename              0
caption               0
Generated          7669
longest_caption       0
dtype: int64

In [311]:

load_dotenv()

OPENAI_API = os.getenv("OPENAI_API")
GEMINI_API = os.getenv("GOOGLE_API")

In [312]:
# Import the Images module from pillow 
from PIL import Image 
  

def encode_image(image_path):
        
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')    

In [313]:
def prompt(captions):
    return """
    You are an expert Bengali question, answer generating assistant. You always generate the question and answer from \
    the given image and the captions to the image. you always have to abide by the conditions that are mentioned below: \n \n  
    CONDITION 1:  The questions are always image and caption aligned and informative \n
    CONDITION 2:  Try to generate answer of one or two words. And the answer must never contain more than three words \n
    CONDITION 3:  Generate the question answer pair in Bengali language \n
        \n \n \
    Here is the caption:\n \n {},\
        \n \nBased on the above captions and below given image, generate ONE Question,Answer pair 
        based on this picture in Bengali language. generate the question answer pair like the following format:
    
    Q# "generated question", A# "generated answer"

    """.format(captions)

In [314]:
def give_q_n_a(image_name, captions):
    # Path to your image
    image_path = f"dataset/chitron/{image_name}"

    # Encode the image
    base64_image = encode_image(image_path)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API}"
    }
    payload = {
        # "model": "gpt-4-vision-preview", "gpt-4-turbo"
        "model": "gpt-4-turbo",
        # "response_format": {"type": "json_object"},
        "messages": [
            {
                "role": "user",
                "content": [
                    # {"type": "text", "text": "Caption the image explaining the contents in it. Use Bengali language to caption the image."},
                    {"type": "text", "text": prompt(captions)},

                    {"type": "image_url", "image_url": {"url": 
    f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": 100
    }   


    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    
    return response.json()#['choices'][0]['message']['content']


In [317]:
import time

sliced_df = df[1000:1100]

In [318]:
# total_tokens = 2520663 #2397670 #1393072 # 325355
# total_input_tokens = 2288098 #2176137 #405867 # 293207
# total_output_tokens = 493624 # 482592 # 1537880 # 293207
#  36.23913 # 3.8965100000000006 $
val = 0

In [319]:
sliced_df.isna().sum()

filename             0
caption              0
Generated          100
longest_caption      0
dtype: int64

In [320]:
for idx, rows in sliced_df.iterrows():
    res = (give_q_n_a(rows['filename'], rows['longest_caption']))

    total_input_tokens = total_input_tokens + res['usage']['prompt_tokens']
    total_output_tokens = total_output_tokens + res['usage']['completion_tokens']
    total_tokens = total_tokens + res['usage']['total_tokens']
    
    df.loc[df['filename'] == str(rows['filename']), 'Generated'] = str(res['choices'][0]['message']['content'])
    
    print("done: ", str(rows['filename']), "input_token: ", res['usage']['prompt_tokens'], "output_token: ", res['usage']['completion_tokens'])
    val = val+1
    time.sleep(.5)
    # break

done:  1002.png input_token:  479 output_token:  65
done:  1003.png input_token:  484 output_token:  47
done:  1004.png input_token:  521 output_token:  49
done:  1005.png input_token:  469 output_token:  59
done:  1006.png input_token:  505 output_token:  44
done:  1007.png input_token:  478 output_token:  40
done:  1008.png input_token:  477 output_token:  55
done:  1009.png input_token:  482 output_token:  43
done:  1010.png input_token:  485 output_token:  76
done:  1011.png input_token:  468 output_token:  43
done:  1012.png input_token:  495 output_token:  64
done:  1013.png input_token:  466 output_token:  44
done:  1014.png input_token:  487 output_token:  50
done:  1015.png input_token:  475 output_token:  58
done:  1016.png input_token:  519 output_token:  60
done:  1017.png input_token:  470 output_token:  56
done:  1018.png input_token:  491 output_token:  53
done:  1019.png input_token:  474 output_token:  45
done:  1020.png input_token:  480 output_token:  60
done:  1021.

KeyboardInterrupt: 

In [292]:
res

{'error': {'message': "You uploaded an unsupported image. Please make sure your image is below 20 MB in size and is of one the following formats: ['png', 'jpeg', 'gif', 'webp'].",
  'type': 'invalid_request_error',
  'param': None,
  'code': 'sanitizer_server_error'}}

In [304]:
sliced_df.head(15)

,filename,caption,Generated,longest_caption
973,975.png,"['কতকগুলো শিশু দাড়িয়ে আছে।', 'কিছু বাচ্চা খেলা...",NaN,কতকগুলো শিশু দাড়িয়ে আছে।
974,976.png,"['একজন নারী চাপকল টিপছে।', 'একজন মহিলা কল চাপছ...",NaN,'একজন মহিলা কল চাপছে আর তার পাশে কিছু মানুষ।
975,977.png,"['কতকগুলো রেলের লাইন আছে।', 'রেল লাইনের পাশে এ...",NaN,'রেল লাইনের পাশে একটি বড় পানির ট্যাঙ্ক।
976,978.png,['জলাশয়ে একজন মাছ ধরছে। দূর দিয়ে একটি নৌকা যাচ...,NaN,জলাশয়ে একজন মাছ ধরছে। দূর দিয়ে একটি নৌকা যাচ্ছে।
977,979.png,['সাদা শার্ট পরিহিত তিনজন বালক পাশাপাশি দাড়িয়ে...,NaN,সাদা শার্ট পরিহিত তিনজন বালক পাশাপাশি দাড়িয়ে আছে।
978,980.png,['দুইজন শিশু বয় পড়ছে। একজন নারী একটি শিশু কোলে...,NaN,'একজন মহিলার কোলে একটি বাচ্চা ও পাশে ২ টি বাচ...
979,981.png,['রাস্তা দিয়ে কতকগুলো শিশু সারিবদ্ধ ভাবে হেঁটে...,NaN,'কিছু বাচ্চা গ্রামের রাস্তা দিয়ে হাতে যাচ্ছে ...
980,982.png,"['সমুদ্রের পারে অনেকগুলো মানুষ আছে।', 'সমদ্র ত...",NaN,সমুদ্রের পারে অনেকগুলো মানুষ আছে।
981,983.png,"['কতকগুলো মানুষ ধান ছাড়াচ্ছে।', '৩ জন পুরুষ মা...",NaN,'৩ জন পুরুষ মানুষ ধান ভাংছে ও পাশে একজন মহিলা।
982,984.png,"['একজন বালিকা বসে পড়ছে। দূরে একজন বসে আছে।', '...",NaN,'মেঝেতে ২ টি বাচ্চা পড়ছে আর চেয়ারে একজন বসে আছে।


In [321]:
print("total input tokens: ", total_input_tokens)
print("total output tokens: ", total_output_tokens)
print("total token count: " , total_tokens)
print(f"total cost: {total_input_tokens * 1e-5 + total_output_tokens * 3e-5}")

total input tokens:  2848053
total output tokens:  556623
total token count:  3143617
total cost: 45.17922


In [322]:
df.isna().sum()

filename              0
caption               0
Generated          7580
longest_caption       0
dtype: int64

In [323]:
df.to_csv("dataset/generated_chitron.csv", index = False)

## Resizing the data 

In [119]:
import os

images = os.listdir("dataset/chitron_original")
len(images)

9154

In [126]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


for i in images:
    image = cv2.imread(f"dataset/chitron_original/{i}")
    try:
        resized = cv2.resize(image, (500, 500))
    except e:
        print(i, e)
    
    cv2.imwrite(f"dataset/chitron/{i}", resized)

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc

True